In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# **Read Data**

In [4]:
read1 = pd.read_csv('bank_data.csv')
df = pd.DataFrame(read1)

inputt = input("please enter percentage of the data needed to be read in % \n").split("%")
percentage_of_data = float(inputt[0])
nrows = int(len(df) * (percentage_of_data/100))

read2 = pd.read_csv('bank_data.csv', nrows=nrows)
data = pd.DataFrame(read2, columns=['age','job','marital','education','housing','y'])

age_bins = [0, 18, 25, 35, 50, 65, 120]
age_labels = ['0-18', '19-25', '26-35', '36-50', '51-65', '65+']

# convert age column to categorical column using the age ranges and labels
data['age'] = pd.cut(data['age'], bins=age_bins, labels=age_labels)

print(data)

please enter percentage of the data needed to be read in % 
100%
        age            job  marital  education housing   y
0     26-35     unemployed  married    primary      no  no
1     26-35       services  married  secondary     yes  no
2     26-35     management   single   tertiary     yes  no
3     26-35     management  married   tertiary     yes  no
4     51-65    blue-collar  married  secondary     yes  no
...     ...            ...      ...        ...     ...  ..
4516  26-35       services  married  secondary     yes  no
4517  51-65  self-employed  married   tertiary     yes  no
4518  51-65     technician  married  secondary      no  no
4519  26-35    blue-collar  married  secondary      no  no
4520  36-50   entrepreneur   single   tertiary     yes  no

[4521 rows x 6 columns]


# **Naive Bayes classifier model**

In [5]:
def calculate_prob_of_lables(df, Y):
    classes = sorted(list(df[Y].unique()))
    prob_of_lables = []
    for i in classes:
        prob_of_lables.append(len(df[df[Y]==i])/len(df))
    return prob_of_lables

In [6]:
def calculate_prob_of_features(df, feat_name, feat_val, Y, label):
    feat = list(df.columns)
    df = df[df[Y]==label]
    p_x_given_y = len(df[df[feat_name]==feat_val]) / len(df)
    return p_x_given_y

In [7]:
def NaiveBayesClassifier(df_train, X_test, Y):
    # get feature names
    features = list(df_train.columns)[:-1]

    # calculate probability of lables
    prob_of_lables = calculate_prob_of_lables(df_train, Y)

    Y_pred = []
    # loop over every data sample
    for x in X_test:
        # calculate probability of features
        labels = sorted(list(df_train[Y].unique()))
        prob_of_features = [1]*len(labels)
        for j in range(len(labels)):  #label length = 2
            for i in range(len(features)):  # 4 features
                prob_of_features[j] *= calculate_prob_of_features(df_train, features[i], x[i], Y, labels[j])

        # calculate the whole probability (p(features) * p(lables))
        whole_prob = [1]*len(labels)
        for j in range(len(labels)):
            whole_prob[j] = prob_of_features[j] * prob_of_lables[j]

        if(np.argmax(whole_prob) == 0):
          Y_pred.append("no")
        else:
          Y_pred.append("yes")

    return np.array(Y_pred)

# **Test Naive Bayes Model**

In [8]:
train, test = train_test_split(data, test_size=.2, random_state=1234)

X_test = test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred = NaiveBayesClassifier(train, X_test, "y")

NB_accuracy = np.sum(Y_pred == Y_test) / len(Y_pred) * 100
print(NB_accuracy , "%")

86.62983425414365 %


# **Decesion Tree classifier model**

In [9]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [10]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            #loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index] != threshold]) # row = [['retired' 'married' 'secondary' 'no' 'no']         row[0] = ['retired' 'admin.' 'self-employed' ... 'blue-collar' 'admin.''blue-collar']
                                                                                                  #   ['admin.' 'married' 'secondary' 'yes' 'no']]
        dataset_right = np.array([row for row in dataset if row[feature_index] == threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child ,mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)

        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain

        return gain
    
    def entropy(self, y_given_feature):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y_given_feature)  # y_given_feature : job = unemployed is yes ---> r_child or no ---> l_child
        entropy = 0
        for label in class_labels:
            p_label = len(y_given_feature[y_given_feature == label]) / len(y_given_feature)   # l_child = low/no , r_child = low/yes ------> low is (feature value)
            entropy += -p_label * np.log2(p_label)
        return entropy

    def gini_index(self, y_given_feature):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y_given_feature)
        gini = 0
        for label in class_labels:
            p_label = len(y_given_feature[y_given_feature == label]) / len(y_given_feature)
            gini += p_label**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        return dataset
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value

        feature_val = x[tree.feature_index]
        if feature_val == tree.threshold:
            return self.make_prediction(x, tree.right)
        else:
            return self.make_prediction(x, tree.left)
            

# **Test Decesion Tree Model**

In [11]:
inputData = data.iloc[:, :-1].values
outputData = data.iloc[:, -1].values.reshape(-1,1)

X_train, X_test, Y_train, Y_test = train_test_split(inputData, outputData, test_size=.2, random_state=1234)

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)

Y_pred = classifier.predict(X_test) 
Y_pred = np.array(Y_pred).reshape(-1, 1)

DT_accuracy = np.sum(Y_pred == Y_test) / len(Y_test) * 100

print(DT_accuracy , "%")

87.18232044198895 %


# **Two classifiers accuracies comparison**

In [12]:
print("Decesion tree classifier accuracy =", DT_accuracy, "%"+ "\n")
print("Naive Bayes classifier accuracy =", NB_accuracy, "%" + "\n")

if(NB_accuracy > DT_accuracy):
  print("Naive Bayes classifier is more accurate than Decesion tree classifier")
elif(NB_accuracy < DT_accuracy):
   print("Decesion tree classifier is more accurate than Naive Bayes classifier")
else:
  print("Both classifiers have the same accuracy")

Decesion tree classifier accuracy = 87.18232044198895 %

Naive Bayes classifier accuracy = 86.62983425414365 %

Decesion tree classifier is more accurate than Naive Bayes classifier
